In [1]:
import pygame
import random
import time

pygame 2.6.0 (SDL 2.28.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize pygame
pygame.init()

# Set up the game window
WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Pickleball Math Smash')

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

FIREWORK_COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]

# Paddle settings
paddle_width = 100
paddle_height = 10
paddle_x = WIDTH // 2 - paddle_width // 2
paddle_y = HEIGHT - 20
paddle_speed = 10

# Ball settings
ball_radius = 10
ball_x = WIDTH // 2
ball_y = HEIGHT // 2
ball_speed_x = 5 * random.choice([-1, 1])
ball_speed_y = 5 * random.choice([-1, 1])

In [3]:
# Scoring system
player_score = 0
computer_score = 0
player_serving = False
game_over = False

# Function to update the score and serve logic
def handle_scoring(correct):
    global player_score, computer_score, player_serving

    if player_serving:
        if correct:
            player_score += 1
        else:
            player_serving = False  # Computer gets the serve
    else:
        if correct:
            player_serving = True  # Player wins back the serve
        else:
            computer_score += 1

    check_game_end()

In [4]:
# Fireworks settings
fireworks_active = False
firework_timer = 0

In [5]:
# Function to handle the coin flip
def coin_flip():
    return random.choice(['player', 'computer'])

# Function to check if someone has won the game
def check_game_end():
    global game_over, fireworks_active, firework_timer
    if (player_score >= 11 or computer_score >= 11) and abs(player_score - computer_score) >= 2:
        game_over = True
        fireworks_active = True
        firework_timer = time.time()  # Start the fireworks timer
        return True
    return False

In [6]:
# Math question settings
current_question = ""
current_answer = ""
user_answer = ""
correct = False
time_limit = 10  # 20 seconds per question

# Function to generate a random math problem with multiple choice answers
def generate_math_problem():
    num1 = random.randint(1, 100)
    num2 = random.randint(1, 100)

    operator = random.choice(['+', '-', '%'])

    if operator == '+':
        question = f"{num1} + {num2}"
        correct_answer = num1 + num2
    elif operator == '-':
        question = f"{num1} - {num2}"
        correct_answer = num1 - num2
    elif operator == '%':
        question = f"What is {num1}% of {num2}?"
        correct_answer = (num1 * num2) // 100

    # Generate two incorrect answers
    wrong_answer1 = correct_answer + random.randint(-10, 10)
    wrong_answer2 = correct_answer + random.randint(-10, 10)

    # Ensure wrong answers are distinct and not equal to the correct answer
    while wrong_answer1 == correct_answer:
        wrong_answer1 = correct_answer + random.randint(-10, 10)
    while wrong_answer2 == correct_answer or wrong_answer2 == wrong_answer1:
        wrong_answer2 = correct_answer + random.randint(-10, 10)

    # Create a list of answers and shuffle them
    choices = [correct_answer, wrong_answer1, wrong_answer2]
    random.shuffle(choices)

    return question, correct_answer, choices

In [7]:
# Coin flip to determine who serves first
first_serve = coin_flip()
if first_serve == 'player':
    player_serving = True

In [8]:
# Start the game loop
running = True
current_question, correct_answer, choices = generate_math_problem()
start_time = time.time()

# Ball movement flag to simulate movement
ball_moving = True

while running:
    screen.fill(BLACK)

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN and not game_over:
            if event.key in [pygame.K_1, pygame.K_2, pygame.K_3]:  # Player picks an option
                user_choice = choices[int(event.unicode) - 1]  # Convert keypress (1, 2, 3) to index
                correct = (user_choice == correct_answer)
                
                # Handle scoring based on correctness and current serve
                handle_scoring(correct)

                # Generate new question after answer submission
                current_question, correct_answer, choices = generate_math_problem()
                start_time = time.time()

    # Check if time limit has passed
    elapsed_time = time.time() - start_time
    if elapsed_time > time_limit:
        # Treat as incorrect answer if time runs out
        handle_scoring(False)
        current_question, correct_answer, choices = generate_math_problem()
        start_time = time.time()

    # Ball movement logic
    if ball_moving:
        ball_x += ball_speed_x
        ball_y += ball_speed_y
    
        # Ball collision with walls
        if ball_x <= ball_radius or ball_x >= WIDTH - ball_radius:
            ball_speed_x *= -1
        if ball_y <= ball_radius or ball_y >= HEIGHT - ball_radius:
            ball_speed_y *= -1

    # Paddle movement
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] and paddle_x > 0:
        paddle_x -= paddle_speed
    if keys[pygame.K_RIGHT] and paddle_x < WIDTH - paddle_width:
        paddle_x += paddle_speed

    # Draw the paddle
    pygame.draw.rect(screen, WHITE, (paddle_x, paddle_y, paddle_width, paddle_height))

    # Draw the ball
    pygame.draw.circle(screen, WHITE, (ball_x, ball_y), ball_radius)

    # Display the math question and multiple-choice answers
    font = pygame.font.Font(None, 36)
    question_text = font.render(f"Question: {current_question}", True, WHITE)
    screen.blit(question_text, (10, 10))
    
    # Display the answer choices (1, 2, 3)
    choice_text_1 = font.render(f"1: {choices[0]}", True, WHITE)
    screen.blit(choice_text_1, (10, 50))
    
    choice_text_2 = font.render(f"2: {choices[1]}", True, WHITE)
    screen.blit(choice_text_2, (10, 90))
    
    choice_text_3 = font.render(f"3: {choices[2]}", True, WHITE)
    screen.blit(choice_text_3, (10, 130))

    # Display the score
    score_text = font.render(f"Drishti: {player_score}  Computer: {computer_score}", True, WHITE)
    screen.blit(score_text, (10, 170))

    # Display the serving status
    serve_text = font.render(f"{'Drishti' if player_serving else 'Computer'} is serving", True, WHITE)
    screen.blit(serve_text, (10, 210))
    
    # Fireworks after game over
    if fireworks_active:
        # Keep fireworks for 10 seconds after the game ends
        if time.time() - firework_timer < 10:
            for _ in range(5):  # Generate 5 random fireworks at a time
                x = random.randint(0, WIDTH)
                y = random.randint(0, HEIGHT)
                color = random.choice(FIREWORK_COLORS)
                radius = random.randint(10, 40)
                pygame.draw.circle(screen, color, (x, y), radius)
        else:
            fireworks_active = False  # Stop fireworks after 10 seconds

    # Check if the game is over
    if game_over:
        winner = "Drishti" if player_score > computer_score else "Computer"
        game_over_text = font.render(f"Game Over! {winner} Wins!", True, WHITE)
        screen.blit(game_over_text, (10, 250))

    # Update display
    pygame.display.flip()

    # Frame rate
    pygame.time.Clock().tick(60)

pygame.quit()